# www.atptour.com Webscraper 

##### To prevent blocking
clicks interval of 0.6 seconds
add randomness

In [79]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

from functions import *

# Webscraper initialization for single page (one date)

In [2]:
url = "https://www.atpworldtour.com/en/rankings/singles/?rankDate=2018-7-30&countryCode=all&rankRange=1-100"
page = requests.get(url)
print(page) 

soup = BeautifulSoup(page.content, "html.parser")

<Response [200]>


## Ranking

In [3]:
ranking = soup.find_all("td", {"class": "rank-cell"})
ranking = [item.get_text().strip() for item in ranking]

### Move

In [4]:
move = soup.find_all("div", {"class": "move-text"})
move = [item.get_text().strip() for item in move]

### Country

In [5]:
country = soup.find_all("div", {"class": "country-item"})
country = [item.find("img", {"alt": True}) for item in country]

country[0:5]

[<img alt="ESP" onerror="this.remove()" src="/en/~/media/images/flags/esp.svg"/>,
 <img alt="SUI" onerror="this.remove()" src="/en/~/media/images/flags/sui.svg"/>,
 <img alt="GER" onerror="this.remove()" src="/en/~/media/images/flags/ger.svg"/>,
 <img alt="ARG" onerror="this.remove()" src="/en/~/media/images/flags/arg.svg"/>,
 <img alt="RSA" onerror="this.remove()" src="/en/~/media/images/flags/rsa.svg"/>]

In [6]:
country_list = []

for item in country:
    current_item = find_between(str(item), '"', '"' )
    country_list.append(current_item)

In [7]:
country_list[0:5]

['ESP', 'SUI', 'GER', 'ARG', 'RSA']

### Player

In [23]:
player = soup.find_all("td", {"class": "player-cell"})
player = [item.get_text().strip() for item in player]

### Age

In [10]:
age = soup.find_all("td", {"class": "age-cell"})
age = [item.get_text().strip() for item in age]

### Points 

In [19]:
points = soup.find_all("a", {"data-ga-label": "rankings-breakdown"})
points = [item.get_text().strip() for item in points]

### Tournaments

In [11]:
tournaments = soup.find_all("td", {"class": "tourn-cell"})
tournaments = [item.get_text().strip() for item in tournaments]

### Point Droppings

In [12]:
points_dropping = soup.find_all("td", {"class": "pts-cell"})
points_dropping = [item.get_text().strip() for item in points_dropping]

### Next Best

In [13]:
next_best = soup.find_all("td", {"class": "next-cell"})
next_best = [item.get_text().strip() for item in next_best]

### Date

In [24]:
date = soup.find_all("div", {"class": "dropdown-label"})
date = date[0].get_text().strip()
date

'2018.07.30'

### Testing

In [25]:
df = pd.DataFrame({
                    "ranking": ranking,
                    "move": move,
                    "country": country_list,
                    "player": player,
                    "age": age,
                    "points": points,
                    "tournaments": tournaments,
                    "points_dropping": points_dropping,
                    "next_best": next_best
                })
df["date"] = date
df.head()

,ranking,move,country,player,age,points,tournaments,points_dropping,next_best,date
0,1,,ESP,Rafael Nadal,32,"9,310",14,0,0,2018.07.30
1,2,,SUI,Roger Federer,36,"7,080",18,0,0,2018.07.30
2,3,,GER,Alexander Zverev,21,"5,665",22,500,45,2018.07.30
3,4,,ARG,Juan Martin del Potro,29,"5,395",19,45,20,2018.07.30
4,5,,RSA,Kevin Anderson,32,"4,655",21,300,0,2018.07.30


# Create Dataframe

### Links

In [3]:
drop_down = soup.find('div', {'class': 'main-content'}).find('ul',{'class': 'dropdown'})

date_list=[]
for i in range(1,len(drop_down.findAll('li'))):
    date_list.append(drop_down.findAll('li')[i].get_text().strip().replace('.','-')) 

len(date_list)

2111

### Scraper testing

In [4]:
urlpattern = "http://www.atpworldtour.com/en/rankings/singles?rankDate={}&rankRange=1-100"
check = scrape('2018-04-23', urlpattern)
check.head()

,ranking,move,country,player,age,points,tournaments,points_dropping,next_best,date
0,1,,ESP,Rafael Nadal,31,"8,770",14,500,0,2018-04-23
1,2,,SUI,Roger Federer,36,"8,670",17,0,0,2018-04-23
2,3,1,GER,Alexander Zverev,21,"5,195",24,0,0,2018-04-23
3,4,1,CRO,Marin Cilic,29,"4,985",20,0,0,2018-04-23
4,5,,BUL,Grigor Dimitrov,26,"4,950",22,0,0,2018-04-23


In [5]:
columns = check.columns.values
df = pd.DataFrame(columns=columns)

In [6]:
counter = 0

for date in tqdm(date_list):
    temp_df = scrape(date, urlpattern)
    df = df.append(temp_df)
    counter += 1

100%|████████████████████████████████████████████████████████████████████████████| 2111/2111 [1:01:46<00:00,  1.56s/it]


In [7]:
df.shape

(202311, 10)

In [12]:
df = df.reset_index(drop=True)
df.head()

,ranking,move,country,player,age,points,tournaments,points_dropping,next_best,date
0,1,,ESP,Rafael Nadal,33,"9,985",17,0,0,2020-01-06
1,2,,SRB,Novak Djokovic,32,"9,055",16,0,0,2020-01-06
2,3,,SUI,Roger Federer,38,"6,590",17,0,0,2020-01-06
3,4,,AUT,Dominic Thiem,26,"5,825",21,0,0,2020-01-06
4,5,,RUS,Daniil Medvedev,23,"5,705",23,0,0,2020-01-06


In [13]:
df.tail()

,ranking,move,country,player,age,points,tournaments,points_dropping,next_best,date
202306,96,,CZE,Vladimir Zednik,26,0,0,0,0,1973-08-23
202307,97,,CHI,Patricio Cornejo,29,0,0,0,0,1973-08-23
202308,98,,HUN,Szabolcz Baranyi,29,0,0,0,0,1973-08-23
202309,98,,CRO,Zeljko Franulovic,26,0,0,0,0,1973-08-23
202310,100,,AUT,Hans Kary,24,0,0,0,0,1973-08-23


In [55]:
df = pd.read_csv('player_rankings.csv', sep=",", encoding='utf8', engine='python')
df.head()

,ranking,move,country,player,age,points,tournaments,points_dropping,next_best,date
0,1,NaN,ESP,Rafael Nadal,33.0,"9,985",17,0,0,2020-01-06
1,2,NaN,SRB,Novak Djokovic,32.0,"9,055",16,0,0,2020-01-06
2,3,NaN,SUI,Roger Federer,38.0,"6,590",17,0,0,2020-01-06
3,4,NaN,AUT,Dominic Thiem,26.0,"5,825",21,0,0,2020-01-06
4,5,NaN,RUS,Daniil Medvedev,23.0,"5,705",23,0,0,2020-01-06


### Write to csv

In [14]:
df.to_csv('player_rankings.csv', encoding='utf-8', index=False)

# Dataframe Cleaning

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202311 entries, 0 to 202310
Data columns (total 10 columns):
ranking            202311 non-null object
move               120751 non-null float64
country            202311 non-null object
player             202311 non-null object
age                202304 non-null float64
points             202311 non-null object
tournaments        202311 non-null int64
points_dropping    202311 non-null object
next_best          202311 non-null int64
date               202311 non-null object
dtypes: float64(2), int64(2), object(6)
memory usage: 15.4+ MB


### Convert columns into int

In [57]:
df['points'] = df['points'].str.replace(r'\W', '')

In [62]:
df['points_dropping'] = df['points_dropping'].str.replace(r'\W', '')

In [64]:
# convert string to integer
int_cols = [i for i in df.columns if i not in ["country","player", "date", "ranking"]]
for col in int_cols:
    df[col]=pd.to_numeric(df[col])

In [71]:
# convert to datetime
df['date'] = pd.to_datetime(df['date'])

In [75]:
df.ranking.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56',
       '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67',
       '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78',
       '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89',
       '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100',
       '65T', '45T', '68T', '94T', '97T', '14T', '60T', '70T', '87T',
       '92T', '61T', '95T', '98T', '62T', '81T', '99T', '67T', '100T',
       '85T', '91T', '58T', '90T', '43T', '54T', '59T', '96T', '66T',
       '64T', '84T', '88T', '13T', '29T', '63T', '53T', '73T', '78T',
       '38T', '93T', '51T', '72T', '27T', '40T', '26T', '52T', '

In [98]:
df.columns[df.isna().any()].tolist()

['move', 'age']

In [100]:
### fill na with 0
df = df.fillna(0)

# Data Analysis

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202311 entries, 0 to 202310
Data columns (total 10 columns):
ranking            202311 non-null object
move               202311 non-null float64
country            202311 non-null object
player             202311 non-null object
age                202311 non-null float64
points             202311 non-null int64
tournaments        202311 non-null int64
points_dropping    202311 non-null int64
next_best          202311 non-null int64
date               202311 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(4), object(3)
memory usage: 15.4+ MB


### How long has Roger Federer been in the top 5?

In [102]:
roger = df[df["player"].str.contains("Federer")]
print(roger.shape)
roger.head()

(1003, 10)


,ranking,move,country,player,age,points,tournaments,points_dropping,next_best,date
2,3,0.0,SUI,Roger Federer,38.0,6590,17,0,0,2020-01-06
102,3,0.0,SUI,Roger Federer,38.0,6590,17,0,0,2019-12-30
202,3,0.0,SUI,Roger Federer,38.0,6590,17,0,0,2019-12-23
302,3,0.0,SUI,Roger Federer,38.0,6590,17,0,0,2019-12-16
402,3,0.0,SUI,Roger Federer,38.0,6590,17,0,0,2019-12-09


In [103]:
roger["ranking"] = roger["ranking"].astype(int)
top5 = roger[roger["ranking"] <= 5]
print('%d Weeks' %len(top5))

773 Weeks


### Who’s the youngest player ever to reach the top 10?

In [104]:
youngest_df = df.copy()
youngest_df['ranking'] = youngest_df['ranking'].str.replace('T','').astype(int)
youngest_df.head()

,ranking,move,country,player,age,points,tournaments,points_dropping,next_best,date
0,1,0.0,ESP,Rafael Nadal,33.0,9985,17,0,0,2020-01-06
1,2,0.0,SRB,Novak Djokovic,32.0,9055,16,0,0,2020-01-06
2,3,0.0,SUI,Roger Federer,38.0,6590,17,0,0,2020-01-06
3,4,0.0,AUT,Dominic Thiem,26.0,5825,21,0,0,2020-01-06
4,5,0.0,RUS,Daniil Medvedev,23.0,5705,23,0,0,2020-01-06


In [109]:
yrankings = youngest_df[(youngest_df["ranking"] <= 10)]
yrankings = yrankings[(yrankings["age"] == min(yrankings["age"]))]
print(yrankings.shape)
yrankings.head()

(72, 10)


,ranking,move,country,player,age,points,tournaments,points_dropping,next_best,date
150198,9,3.0,USA,Michael Chang,17.0,0,0,0,0,1990-02-19
150295,6,1.0,USA,Michael Chang,17.0,0,0,0,0,1990-02-12
150394,5,0.0,USA,Michael Chang,17.0,0,0,0,0,1990-02-05
150494,5,1.0,USA,Michael Chang,17.0,0,0,0,0,1990-01-29
150595,6,0.0,USA,Michael Chang,17.0,0,0,0,0,1990-01-22


In [110]:
yrankings.groupby("player").agg({"date": "min", "age": "min"}).reset_index().sort_values("date")

,player,date,age
1,Bjorn Borg,1974-06-03,17.0
3,Mats Wilander,1982-07-12,17.0
0,Aaron Krickstein,1984-08-13,17.0
2,Boris Becker,1985-07-08,17.0
4,Michael Chang,1989-06-12,17.0


### What’s the most catastrophic ranking drop a player ever suffered from one year to another?

In [117]:
rankdrop_df = df.copy()
rankdrop_df.index = rankdrop_df["date"]
rankdrop_df = rankdrop_df.drop(columns=['date'])

rankdrop_df.head()

,ranking,move,country,player,age,points,tournaments,points_dropping,next_best
date,,,,,,,,,
2020-01-06,1,0.0,ESP,Rafael Nadal,33.0,9985,17,0,0
2020-01-06,2,0.0,SRB,Novak Djokovic,32.0,9055,16,0,0
2020-01-06,3,0.0,SUI,Roger Federer,38.0,6590,17,0,0
2020-01-06,4,0.0,AUT,Dominic Thiem,26.0,5825,21,0,0
2020-01-06,5,0.0,RUS,Daniil Medvedev,23.0,5705,23,0,0


In [118]:
rankdrop_df['ranking'] = rankdrop_df['ranking'].str.replace('T','').astype(int)

In [119]:
 rankdrop_df = rankdrop_df.groupby(["player", rankdrop_df.index.year]).agg({"ranking": "mean"}).reset_index()

In [120]:
rankdrop_df.head()

,player,date,ranking
0,Aaron Krickstein,1983,95.000000
1,Aaron Krickstein,1984,42.840909
2,Aaron Krickstein,1985,18.243902
3,Aaron Krickstein,1986,32.243902
4,Aaron Krickstein,1987,35.372093


In [121]:
rankdrop_df["shift"] = rankdrop_df["date"] - rankdrop_df["date"].shift(1)
rankdrop_df.head()

,player,date,ranking,shift
0,Aaron Krickstein,1983,95.000000,NaN
1,Aaron Krickstein,1984,42.840909,1.0
2,Aaron Krickstein,1985,18.243902,1.0
3,Aaron Krickstein,1986,32.243902,1.0
4,Aaron Krickstein,1987,35.372093,1.0


In [122]:
for i in range(1, len(rankdrop_df)):
    if rankdrop_df.loc[i, "shift"] == 1:
        try:
            rankdrop_df.loc[i, 'ranking_diff'] = rankdrop_df.loc[i, 'ranking'] - rankdrop_df.loc[i-1, 'ranking']
        except:
            rankdrop_df.loc[i, 'ranking_diff'] = np.nan
    else:
        rankdrop_df.loc[i, 'ranking_diff'] = np.nan
        
rankdrop_df.head()

,player,date,ranking,shift,ranking_diff
0,Aaron Krickstein,1983,95.000000,NaN,NaN
1,Aaron Krickstein,1984,42.840909,1.0,-52.159091
2,Aaron Krickstein,1985,18.243902,1.0,-24.597007
3,Aaron Krickstein,1986,32.243902,1.0,14.000000
4,Aaron Krickstein,1987,35.372093,1.0,3.128191


In [124]:
rankdrop_df[rankdrop_df["ranking_diff"] == rankdrop_df["ranking_diff"].max()]

,player,date,ranking,shift,ranking_diff
5522,Robert Lutz,1982,95.0,1.0,67.5
